# iEEG Source‑Estimation Pipeline

Walk‑through with **OpenNeuro dataset `ds003848` (subject `sub‑RESP0521`)**.

This notebook demonstrates **each function** in the pipeline script you are working on. Run the cells sequentially; every step is self‑contained and heavily commented to make the logic clear.

## 0. Set‑up

*If you are running this on Colab or a fresh environment, uncomment the next cell to install dependencies*.

In [ ]:
# !pip install --quiet mne nibabel nilearn openneuro-py pandas numpy


In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import mne
import nibabel as nib
from nilearn import plotting

# Make sure the pipeline script (patched version) is on PYTHONPATH or in same folder.
# Here we assume it is saved as `ieeg_pipeline.py`
from ieeg_pipeline import (
    PipelineConfig,
    load_raw,
    load_electrodes,
    set_montage_and_types,
    make_subject_id,
    build_forward_model,
    create_evoked_or_epochs,
    estimate_sources,
    save_nifti,
)

print("MNE version:", mne.__version__)


## 1. Download sample data from OpenNeuro

We will grab **one session** (`ses‑1`) of `sub‑RESP0521`.
Download only the iEEG recording (`.vhdr/.eeg/.vmrk`), the electrodes table, and the T1‑weighted MRI.

If you already have these files locally, point `data_root` to that directory and skip the download.

In [ ]:
from openneuro import download

data_root = Path("data_openneuro")
if not data_root.exists():
    data_root.mkdir()

dataset_id = "ds003848"

# Uncomment to download (~200 MB)
# download(
#     dataset=dataset_id,
#     target_dir=data_root,
#     include=[
#         "sub-RESP0521/ses-1/ieeg/*_ieeg.vhdr",
#         "sub-RESP0521/ses-1/ieeg/*_ieeg.eeg",
#         "sub-RESP0521/ses-1/ieeg/*_ieeg.vmrk",
#         "sub-RESP0521/ses-1/ieeg/sub-RESP0521_ses-1_electrodes.tsv",
#         "sub-RESP0521/anat/sub-RESP0521_T1w.nii.gz",
#     ],
#     verbose=True,
# )


### Define paths

In [ ]:
sub = "sub-RESP0521"
ses = "ses-1"
ieeg_file = next((data_root / sub / ses / "ieeg").glob("*_ieeg.vhdr"))
electrodes_tsv = data_root / sub / ses / "ieeg" / f"{sub}_{ses}_electrodes.tsv"
mri_path = data_root / sub / "anat" / f"{sub}_T1w.nii.gz"

print("iEEG file :", ieeg_file)
print("Electrodes:", electrodes_tsv)
print("T1 MRI    :", mri_path)


## 2. Prepare electrodes CSV (x,y,z,name,type)
The pipeline expects **comma‑separated** values with at least `x,y,z`.

In [ ]:
csv_path = electrodes_tsv.with_suffix('.csv')
if not csv_path.exists():
    df = pd.read_csv(electrodes_tsv, sep='\t')
    # Keep only good rows where x/y/z are finite
    df_valid = df[np.isfinite(df['x']) & np.isfinite(df['y']) & np.isfinite(df['z'])]
    cols = ['name', 'x', 'y', 'z']
    if 'hemisphere' in df_valid.columns:
        cols.append('hemisphere')  # example extra col
    df_valid[cols].to_csv(csv_path, index=False)
print("Saved CSV to", csv_path)

## 3. Create a `PipelineConfig`

In [ ]:
cfg = PipelineConfig(
    ieeg_path=ieeg_file,
    electrodes_path=csv_path,
    mri_path=mri_path,
    subjects_dir=None,          # fall back to fsaverage + spherical head‑model
    spacing_mm=6.0,
    inverse_method="dSPM",
    snr=3.0,
    time_window=(0, 30),        # first 30 s as example
    coord_units="mm",         # TSV provides mm
    keep_time=False,
    to_mni=True,                # morph to fsaverage/MNI
    verbose=False,
)
cfg

### 3.1 `load_raw`

In [ ]:
raw = load_raw(cfg)
print(raw)
print("Data shape:", raw.get_data().shape)

### 3.2 `load_electrodes`

In [ ]:
coords, names, types = load_electrodes(cfg, n_channels=len(raw.ch_names))
print("coords shape:", coords.shape)
print("first 5 coords (m):\n", coords[:5])
print("sample names:", names[:5])
print("types counts:", pd.Series(types).value_counts().to_dict())

### 3.3 `set_montage_and_types`

In [ ]:
set_montage_and_types(raw, coords, names, types)
print(">> Montage set. Dig points:", len(raw.info['dig']))

### 3.4 `make_subject_id` & `build_forward_model`

In [ ]:
subject = make_subject_id(cfg)
fwd, src = build_forward_model(cfg, raw, subject)
print(f"Subject inferred: {subject}")
print(f"Forward solution with {fwd['nsource']} sources and {fwd['nchan']} channels.")

### 3.5 `create_evoked_or_epochs`

In [ ]:
data_obj = create_evoked_or_epochs(raw, cfg)
print(type(data_obj), data_obj)

### 3.6 `estimate_sources` (full pipeline wrapper)

In [ ]:
zmap, affine = estimate_sources(cfg)
print("Z‑map shape:", zmap.shape)

### 3.7 `save_nifti` & quick visualisation

In [ ]:
nii_path = Path("ieeg_sources_resp0521_z.nii.gz")
save_nifti(zmap, affine, nii_path)
print("Saved to", nii_path)

# Quick interactive view (Nilearn). Commented for headless servers.
# plotting.view_img(nii_path, threshold=3).open_in_browser()


## 4. Summary
* We loaded a BIDS‑formatted OpenNeuro iEEG dataset.
* Converted electrodes.tsv to a simple CSV.
* Walked through every helper in the pipeline and produced a volumetric Z‑scored source map in MNI space.*